https://adventofcode.com/2017/day/10

In [188]:
data = "187,254,0,81,169,219,1,190,19,102,255,56,46,32,2,216"
puzzle_input = [int(x) for x in data.split(",")]
puzzle_input

[187, 254, 0, 81, 169, 219, 1, 190, 19, 102, 255, 56, 46, 32, 2, 216]

In [201]:
import operator
import toolz

In [211]:
class CircularList(list):
    """Thanks to https://stackoverflow.com/a/47606550/203629

    N.B. Negative indices may produce unexpected results.
    """

    def __getitem__(self, key):
        if isinstance(key, slice):
            return [self[x] for x in self._rangeify(key)]

        key = operator.index(key)
        if key < 0:
            raise IndexError("Negative indices not defined for CircularList")
        try:
            return super().__getitem__(key % len(self))
        except ZeroDivisionError:
            raise IndexError("Can't get an item from an empty CircularList")

    def __setitem__(self, key, value):
        if isinstance(key, slice):
            for k, v in zip(self._rangeify(key), value):
                self[k] = v
            return

        key = operator.index(key)
        if key < 0:
            raise IndexError("Negative indices not defined for CircularList")
        super().__setitem__(key % len(self), value)

    def _rangeify(self, slice_):
        start, stop, step = slice_.start, slice_.stop, slice_.step
        if start is None:
            start = 0
        if stop is None:
            stop = start + len(self)
        if step is None:
            step = 1
        if start < 0 or stop < 0:
            raise IndexError("Negative indices not defined for CircularList")
        return range(start, stop, step)


In [212]:
class Knotter:
    def __init__(self, lengths=puzzle_input):
        self.lengths = lengths
        self.pos = 0
        self.skip = 0
        self.cl = CircularList(range(256))
        self.size = len(self.cl)

    def knot(self, length):
        cl = self.cl
        pos = self.pos
        cl[pos:pos+length] = cl[pos:pos+length][::-1]
        self.pos = (pos + length + self.skip) % self.size
        self.skip += 1

    def round(self):
        for length in self.lengths:
            self.knot(length)



In [213]:
%time
kr = Knotter()
kr.round()

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 20 µs


Part 1

In [214]:
kr.cl[0] * kr.cl[1]

1980

Part 2

In [192]:
class Knotter2(Knotter):
    def run(self):
        for _ in range(64):
            self.round()

In [190]:
input_2 = [ord(c) for c in data] + [17, 31, 73, 47, 23]

In [193]:
%%time
kr2 = Knotter2(input_2)
kr2.run()

CPU times: user 191 ms, sys: 0 ns, total: 191 ms
Wall time: 188 ms


In [196]:
' '.join(str(x) for x in kr2.cl)

'93 3 82 45 157 35 27 216 64 200 44 60 58 158 208 57 150 65 155 180 131 56 253 237 238 246 130 168 189 135 241 59 171 199 117 164 124 226 84 201 227 69 207 254 178 230 213 140 101 196 218 9 86 172 163 176 243 162 29 142 81 147 96 33 245 50 221 55 211 255 203 161 215 210 170 175 240 71 119 105 127 22 134 25 83 113 92 102 154 152 139 98 118 222 185 4 77 153 156 247 146 160 202 54 114 91 67 205 110 239 250 191 85 31 34 229 167 15 179 11 192 235 242 87 1 115 244 126 38 32 198 26 107 129 80 78 138 145 70 177 48 6 251 204 97 39 12 214 194 173 249 120 151 103 187 95 149 148 75 2 41 193 17 225 137 37 14 53 63 252 89 136 143 51 184 182 159 181 188 133 197 228 112 125 195 128 99 7 13 108 36 16 30 21 141 169 165 116 183 42 5 62 61 76 79 47 74 18 66 43 104 72 106 212 8 90 144 223 190 186 219 209 233 231 28 73 123 68 52 10 46 0 23 248 94 88 121 122 111 19 206 24 132 217 109 224 20 174 220 234 236 40 49 232 100 166'

In [205]:
dense_hash = [toolz.reduce(operator.xor, x) for x in toolz.partition_all(16, kr2.cl)]
dense_hash

[137, 145, 36, 218, 194, 16, 18, 235, 195, 46, 47, 77, 17, 234, 236, 85]

In [210]:
"".join([f"{i:0>2x}" for i in dense_hash])

'899124dac21012ebc32e2f4d11eaec55'